In [32]:
experiment_name = "uci_all"
results_dir = './results/' + experiment_name + '/'

In [3]:
import seaborn as sns
sns.set_theme(style="ticks")
sns.set_context("poster")

import matplotlib.pyplot as plt
import pandas as pd
import pickle 
import os 

import numpy as np
import pandas
    
import runs.config_experiments_uci_all as run
experiment_list = run.config_experiments(results_dir, create_json=False)

36064 config files created


In [4]:
if not os.path.isfile('./results/datasets/UCI/datasets_info.csv'):
    import json
    datasets = []
    for data_set in range(20,66):
        with open('/vast/robustness/configs_datasets/' + str(data_set) + '.json') as config_file:
            config = json.load(config_file)
            tmpX = np.shape(np.genfromtxt('/vast/robustness/datasets/UCI/imp_' + config["name_file"] 
                                 + '_' + "trainX.csv", delimiter=','))
            config["num_examples"] = tmpX[0]
            config["num_features"] = tmpX[1]
            datasets.append(config)
    df = pd.DataFrame(datasets) 
    df.to_csv('./results/datasets/UCI/datasets_info.csv')
else:
    df = pd.read_csv('./results/datasets/UCI/datasets_info.csv')

name_attacks = ["l1_pgd_norm", "l1_fgm_norm","l2_pgd_norm", "l2_fgm_norm", "linf_pgd","l2_pgd", "linf_fgsm", "l2_fgm"]#


In [5]:
df[(df.num_features>50) & (df.num_examples > 500)]

,Unnamed: 0,dataset_id,dataset_name,name_file,num_classes,num_examples,num_features
0,0,20,UCI,ozone-level-detection-eight,2,2027,73
4,4,24,UCI,optical-recognition-handwritten-digits,10,3058,65
15,15,35,UCI,spambase,2,3681,58
16,16,36,UCI,ozone-level-detection-one,2,2029,73
41,41,61,UCI,cnae-9,9,864,857


In [15]:
df.to_csv('dataset.csv',index=False)

In [23]:
approxs = ['linf', 'l1'] 
test = {}
for approx in approxs:
    file_name = results_dir + experiment_list[1]['model_name'] + '/results/acc_' + 'test' + '_approx_bound_' + approx + '.pkl'
    if not os.path.isfile(file_name):
        print("Missing!! " + file_name)
        continue

    with open(file_name, 'rb') as f:
        tmp = pickle.load(f)
            
    test[approx] = {}
    for epsilon in tmp['bound'].keys():
        test[approx][epsilon] = {}
        test[approx][epsilon]['test'] = 0
        test[approx][epsilon]['total'] = 0
        
for idx, _ in enumerate(experiment_list):
    for approx in approxs:
        file_name = results_dir + experiment_list[idx]['model_name'] + '/results/acc_' + 'test' + '_approx_bound_' + approx + '.pkl'
        if not os.path.isfile(file_name):
            print("Missing!! " + file_name)
            continue

        with open(file_name, 'rb') as f:
            tmp = pickle.load(f)
            
        for epsilon in tmp['bound'].keys():
            if np.isnan(tmp['bound'][epsilon]) or np.isnan(tmp['xent'][epsilon]):
                continue
            test[approx][epsilon]['total'] += 1
            if tmp['bound'][epsilon] >= tmp['xent'][epsilon]:
                test[approx][epsilon]['test'] += 1
for approx in approxs:
    for epsilon in tmp['bound'].keys():
        test[approx][epsilon] = test[approx][epsilon]['test'] /  test[approx][epsilon]['total']     

In [30]:
df_approx = pd.DataFrame.from_dict(test, orient='index')
df_approx.to_csv('bound_approx.csv', index=False)
print(df_approx)

        0.0000    0.0002    0.0005    0.0008    0.0010    0.0015    0.0020  \
linf  0.935609  0.986844  0.987781  0.987490  0.988072  0.987846  0.986941   
l1    0.934030  0.989749  0.990493  0.990816  0.990945  0.990978  0.991042   

        0.0030    0.0100    0.1000    0.3000    0.5000    1.0000  
linf  0.987296  0.984322  0.948991  0.862199  0.803142  0.786527  
l1    0.991366  0.991818  0.990816  0.989458  0.987162  0.981891  


In [19]:

list_entries = []
for net in ['OneLayer', 'ThreeLayer']:
    for dataset_id in list(df.dataset_id):

        for idx, exp in enumerate(experiment_list):
            if exp['data_set'] == dataset_id: 
                break
                
        for attack in name_attacks:

            file_name = results_dir + experiment_list[idx]['model_name'] + '/results/acc_' + 'val' + '_' + attack + '.pkl'
            if not os.path.isfile(file_name):
                print("Missing!! " + file_name)
                continue
            with open(file_name, 'rb') as f:
                tmp = pickle.load(f)


            for cv_epsilon in list(tmp.keys()):
                # Hash table of parameters
                parameters = {"epsilon": {}, "backbone": {}, "initial_learning_rate": {}, 
                              "robust_training": {}, "type_robust": {}, "epsilon_pgd_training":{}}

                to_exclude = []
                experiment_list_tmp = [element for i, element in enumerate(experiment_list) if i not in to_exclude]
                for exp in experiment_list_tmp:
                    if not exp['data_set'] == dataset_id: 
                        continue
                    for kk in parameters.keys():
                        if exp[kk] in parameters[kk]:
                            parameters[kk][exp[kk]].append(int(exp["model_name"]))
                        else:
                            parameters[kk][exp[kk]] = [int(exp["model_name"])]


                # For all methods, do cross-val and create an entry of the results
                backbones = [net, net + '+pgd']



                for backbone in backbones:
                    for robust_training in [True, False]:
                        if robust_training:
                            type_robust_trainings = ['l1','linf', "certificate", "grad"] 
                        else:
                            type_robust_trainings = ['none']
                        for type_robust in type_robust_trainings:

                            if (backbone == 'Madry' and robust_training == True) or \
                                (backbone == 'CNN+clipping' and robust_training == False):
                                continue

                            if robust_training==False:
                                ids = list(set(parameters["backbone"][backbone]) & 
                                           set(parameters["robust_training"][False]))
                            else:
                                ids = list(set(parameters["backbone"][backbone]) & 
                                       set(parameters["robust_training"][True])&
                                      set(parameters["type_robust"][type_robust]))



                            if backbone == net + '+pgd' and robust_training == True:
                                continue

                            if ids == []:
                                continue
                            #print(ids)


                            # Cross-validation among learning rates and epsilons:
                            best_acc = -1
                            best_id = ids[0]
                            for id in ids:
                                file_name = results_dir + experiment_list[id]['model_name'] + '/results/acc_' + 'val' + '_' + attack + '.pkl'
                                if not os.path.isfile(file_name):
                                    print("Missing!! " + file_name)
                                    continue

                                with open(file_name, 'rb') as f:
                                    tmp = pickle.load(f)

                                acc = tmp[cv_epsilon]
                                if acc>best_acc:
                                    best_id = id
                                    best_acc = acc

                            if best_acc == -1:
                                continue

                            if (robust_training == False) & (backbone==net):  
                                name_legend = 'vanilla'
                            elif backbone== net + '+pgd':
                                name_legend = 'pgd'
                            else:
                                if type_robust=='certificate':
                                    name_legend = 'RUB'
                                elif type_robust=='linf':
                                    name_legend = 'aRUB_Linf'
                                elif type_robust=='grad':
                                    name_legend = 'grad'
                                else:
                                    name_legend = 'aRUB_L1'

                            entry = {"dataset": dataset_id,
                                     "net": net,
                                    "learning_rate": experiment_list[best_id]['initial_learning_rate'],
                                    "robust_training": name_legend,
                                    "epsilon": experiment_list[best_id]['epsilon'],
                                    "epsilon_pgd_training": experiment_list[best_id]['epsilon_pgd_training']}

                            dataset = "test"
                            entry["attack"] = attack
                            entry["experiment_id"] = best_id

                            with open(results_dir + experiment_list[best_id]['model_name'] + '/results/acc_' + dataset + '_' + 
                                attack + '.pkl', 'rb') as f:
                                tmp = pickle.load(f)

                            entry["test_epsilon"] = cv_epsilon
                            entry["accuracy"] =  100*tmp[cv_epsilon]
                            
                            with open(results_dir + experiment_list[best_id]['model_name'] + 
                                          '/results/training_time.pkl', 'rb') as f:
                                tmp = pickle.load(f)
                                entry["images_per_second"] = np.mean(tmp) 
                                entry["std_images_per_second"] = np.std(tmp)
                            
                            list_entries.append(entry.copy())

df_results = pd.DataFrame.from_dict(list_entries) 


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [26]:
df_results.to_csv('uci_norm.csv', index=False)